In [ ]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))


# Any results you write to the current directory are saved as output.

This kernel is specifically is for Beginners who want's to experiment building CNN using Keras. By using this kernel, you can expect to get good score and also learn keras. 
Keras is simple frameworks where we can initialize the model and keep stacking the layers we want. It makes building deep neural networks very easy.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from os.path import join as opj
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [2]:
#Load the data.
train = pd.read_json("../input/train.json")

In [3]:
#test = pd.read_json("../input/test.json")

In [4]:
train['band_1'] = [(np.array(img).reshape(75,75) - \
                    np.array(img).reshape(75,75).min())/(np.array(img).reshape(75,75).max() - \
                                                         np.array(img).reshape(75,75).min()) \
                   for img in train['band_1']]

In [5]:
#Generate the training data
#Create 3 bands having HH, HV and avg of both
X_band_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75, 1) for band in train["band_1"]])
X_train = X_band_1
#X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
#X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)

Keras CNN implementation

In [6]:
#Import Keras.
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

Using TensorFlow backend.


In [7]:
#CNN CLASSIFIER DEFINITION
def getModel():
    #Building the model
    gmodel=Sequential()
    #Conv Layer 1
    gmodel.add(Conv2D(64, kernel_size=(5, 5), activation='relu', padding='same', input_shape=(75, 75, 1)))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))) #yields (75-2)/2 + 1 = 37 edges
    gmodel.add(Dropout(0.2))

    #Conv Layer 1_1
    gmodel.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))) #yields (37-2)/2 + 1 = 18 edges
    gmodel.add(Dropout(0.2))

    #Conv Layer 1_2
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))) #yields (18-2)/2 + 1 = 9 edges
    gmodel.add(Dropout(0.2))

    #Conv Layer 1_3
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1))) #yields (9-2)/1 + 1 = 4 edges
    gmodel.add(Dropout(0.2))
    
    #Conv Layer 2
    #gmodel.add(Conv2D(256, kernel_size=(6, 6), strides=3, padding='valid', activation='relu'))
    #gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
    #gmodel.add(Dropout(0.2))
    
    #Flatten the data for upcoming dense layers
    gmodel.add(Flatten())

    #Dense Layers
    gmodel.add(Dense(512))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Dense Layer 2
    gmodel.add(Dense(256))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Sigmoid Layer
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))

    myoptim = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.0) #epsilon=1e-08
    gmodel.compile(loss='binary_crossentropy',
                  optimizer=myoptim,
                  metrics=['accuracy'])
    gmodel.summary()
    return gmodel


def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)


Keeping the same tt ratio as in the CapsNet NB

In [12]:
target_train=train['is_iceberg']
X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train, random_state=42, train_size=0.92)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [13]:
#Without denoising, core features.
import os
gmodel=getModel()
gmodel.fit(X_train_cv, y_train_cv,
          batch_size=100,
          epochs=20,
          verbose=1,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 75, 75, 64)        1664      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 37, 37, 64)        36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 18, 18, 64)        0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 18, 18, 128)       73856     
__________

Evaluating Test Accuracy -- Not Yet Performed

In [ ]:
#gmodel.load_weights(filepath=file_path)
#score = gmodel.evaluate(X_valid, y_valid, verbose=1)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])

In [ ]:

#X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
#X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
#X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
#                          , X_band_test_2[:, :, :, np.newaxis]
#                         , ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)
#predicted_test=gmodel.predict_proba(X_test)

In [ ]:
#submission = pd.DataFrame()
#submission['id']=test['id']
#submission['is_iceberg']=predicted_test.reshape((predicted_test.shape[0]))
#submission.to_csv('sub.csv', index=False)